<a href="https://colab.research.google.com/github/yskmt2018/quantum2021/blob/main/spot_lecture_assign_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# スポット講義割り当て問題 - Spot Lecture Assign Problem
- This notebook was created and tested on Google Colaboratory. (2021/06)

In [1]:
!python -V
!pip install --quiet pyqubo==1.0.12 openjij==0.3.5

Python 3.7.10


In [2]:
import re
import numpy as np
import pandas as pd
import pyqubo as pq
import openjij as oj
from IPython.display import display_html
np.random.seed(0)

## 目次

1. 目的
2. 問題設定
3. 定式化
4. Hamiltonian & QUBO
5. (疑似)量子アニーリング
6. 可視化
7. 振り返り・展望

## 1. 目的

- スポット講義における「講師の割り当て」問題について、組合せ最適化問題として定式化し、(疑似)量子アニーリングを用いて解を得る。

## 2. 問題設定

今回は、以下の条件を適用する。

- 講師数、講義数、各講義の人数は、2020年度スポット講義の実績値を利用する。
- 各講師は、第1～第3希望を出す。できるだけ高い希望の講義に選ばれるようにする。
- 各講師は、希望しない講義を2つ出す。希望しない講義に選ばれてはならない。
- 各講師は、2つ以上の講義を受け持ってはならない。また、講師全員が何らかの講義を担当しなくてはならない。
- 各講義の定員を満たさなくてはならない。

In [3]:
# 講師数(2020年度実績)
Tn = 38
# 講義数(2020年度実績)
Sn = 9

In [4]:
# 講師のリスト
T = list(range(Tn))
print(T)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]


In [5]:
# 講義のリスト
S = list(range(Sn))
print(S)

[0, 1, 2, 3, 4, 5, 6, 7, 8]


In [6]:
# 各講義の参加人数
Sn_s = np.full(shape=Sn, fill_value=4, dtype=np.int)
# 人数が4名の講義が7個、5名の講義が2個(2020年度実績)
Sn_s[3] = 5
Sn_s[7] = 5
print(Sn_s)

[4 4 4 5 4 4 4 5 4]


In [7]:
# 各講師の希望する講義をランダム選択: 第1-3希望に3-1点, その他:-1点
C_ts = np.full(shape=(Tn, Sn), fill_value=-1, dtype=np.int)
for t in T:
  # 希望する講義を S から 3 個ランダムに選択する。replace=False で重複なし
  hopes = np.random.choice(S, size=3, replace=False)
  C_ts[t][hopes] = [3, 2, 1]
print(C_ts)

[[-1  1  2 -1 -1 -1 -1  3 -1]
 [-1  2 -1  3 -1 -1 -1  1 -1]
 [-1 -1  2  1 -1  3 -1 -1 -1]
 [-1 -1  3 -1 -1 -1  2  1 -1]
 [-1 -1 -1 -1  1  3 -1  2 -1]
 [-1 -1  3 -1 -1 -1 -1  1  2]
 [ 1  3 -1 -1 -1 -1 -1 -1  2]
 [-1  1  2  3 -1 -1 -1 -1 -1]
 [-1 -1 -1  1 -1  2 -1 -1  3]
 [-1  2 -1 -1  3  1 -1 -1 -1]
 [ 1 -1 -1 -1 -1 -1  3 -1  2]
 [-1  3  2 -1 -1  1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1  2  1  3]
 [-1 -1 -1 -1  3 -1  1 -1  2]
 [-1 -1 -1 -1  3 -1  2 -1  1]
 [-1 -1 -1 -1  1 -1  3  2 -1]
 [ 3 -1 -1 -1 -1  2 -1 -1  1]
 [ 3 -1 -1 -1 -1  2 -1  1 -1]
 [-1 -1 -1 -1 -1 -1  1  3  2]
 [-1  2 -1 -1  1  3 -1 -1 -1]
 [-1 -1  1  2 -1 -1 -1  3 -1]
 [-1  3 -1 -1 -1 -1 -1  1  2]
 [-1 -1 -1 -1 -1  3 -1  2  1]
 [ 3  1 -1 -1 -1 -1  2 -1 -1]
 [-1 -1  3  1  2 -1 -1 -1 -1]
 [-1 -1  2 -1  3 -1 -1 -1  1]
 [-1  1  3 -1  2 -1 -1 -1 -1]
 [ 2 -1  1 -1 -1 -1 -1  3 -1]
 [-1  3 -1 -1  1 -1  2 -1 -1]
 [-1  3  2 -1  1 -1 -1 -1 -1]
 [ 1  3  2 -1 -1 -1 -1 -1 -1]
 [ 3 -1  2 -1 -1 -1  1 -1 -1]
 [ 2 -1  1 -1  3 -1 -1 -1 -1]
 [ 3 -1 -1

In [8]:
# 各講師の希望しない講義をランダム選択: 対象の講義に1, それ以外:0
NG_ts = np.zeros(shape=(Tn, Sn), dtype=np.int)
for t in T:
  # 希望しない講義を C_ts で -1 となっている講義から 2 個ランダムに選択する。replace=False で重複なし
  ng = np.random.choice(np.where(C_ts[t] == -1)[0], size=2, replace=False)
  NG_ts[t][ng] = [1, 1]
print(NG_ts)

[[1 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 1]
 [0 1 0 0 1 0 0 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 1 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0]
 [0 1 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1]
 [0 0 0 1 1 0 0 0 0]
 [0 1 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1]
 [1 0 0 0 0 1 0 0 0]
 [0 1 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 1]
 [0 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0]
 [1 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 1 0 0 1 0 0 0 0]
 [0 1 0 0 0 1 0 0 0]
 [0 0 1 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 1]
 [0 0 0 1 0 0 1 0 0]
 [0 0 0 0 0 1 1 0 0]
 [0 0 0 1 1 0 0 0 0]]


## 3. 定式化

- 「目的関数」と「制約項」を作成する。
- 目的関数が「最大(最小)化する指標」を、制約項が「遵守させる条件」を表現する。
- 制約項は、制約違反がない場合0を返す。0でない場合、ハミルトニアン(後述)のエネルギーが極端に大きく(不安定な状態に)なるため、制約を守るように作用する。

### 目的関数

- 各講師の希望(を元にした点数の総和)の最大化を目指す。
 - 各講師の点数が C [ t ][ s ] * x [ t ][ s ] であり、その総和(全員の点数の合計)を算出する。
 - 目的関数はコスト関数とも呼ばれ、この関数の出力が小さいほど優れた解である。そのため、最大化問題の際には結果に-1を乗し、優れた解であるほど出力を小さくする必要がある。

$$
maximize \sum_{t \in_{T}, s \in_{S}} (C_{ts} * x_{ts})
$$

In [9]:
def objective(x):
  H = sum(C_ts[t][s] * x[t][s] for t in T for s in S)
  # 最大化のため
  return H * (-1)

### 制約項 1. 受け持つ講義は1つ

- 各講師の受け持つ講義は1つであることを課す。
  - 各講師の受け持つ講義の数が sum( x [ t ][ s ] for s in S ) であり、全ての講師 t∈T について、この数が1でなくてはならない。

$$
\forall_{t \in_{T}}, \sum_{s \in_{S}} x_{ts} = 1
$$

In [10]:
def constraint_onlyone(x):
  # 制約違反の場合、必ず正の値を返す必要があるため、2乗
  return sum(
      (sum(x[t][s] for s in S) - 1)**2 for t in T
  )

### 制約項 2. 定員

- 各講義の定員が満たされることを課す。
  - 各講義の参加人数が sum( x [ t ][ s ] for t in T ) であり、全ての講義 s∈S について、この数が各講義の参加人数( Sn [ s ] )と一致しなくてはならない。 

$$
\forall_{s \in_{S}}, \sum_{t \in_{T}} x_{ts} = Sn_{s}
$$

In [11]:
def constraint_capacity(x):
  # 制約違反の場合、必ず正の値を返す必要があるため、2乗
  return sum(
      (sum(x[t][s] for t in T) - Sn_s[s])**2 for s in S
  )

### 制約項 3. 希望しない講義に割り当てない

- 各講師を希望しない講義に割り当てないことを課す。
  - NG [ t ][ s ] * x [ t ][ s ] は、希望しない講義に割り当てられた場合1となる。全ての講師 t∈T について、この数が0でなくてはならない。

$$
\forall_{t \in_{T}}, \sum_{s \in_{S}} NG_{ts} * x_{ts} = 0
$$

In [12]:
def constraint_unwanted(x):
  return sum(
      sum(NG_ts[t][s] * x[t][s] for s in S) for t in T
  )

## 4. Hamiltonian & QUBO

- ライブラリを利用して、Hamiltonian(ハミルトニアン)を構築し、QUBO(キューボ)に変換する。ここでは、【[PyQUBO](https://github.com/recruit-communications/pyqubo)】を利用する。
- 利用できるライブラリとして、他に【[JijModeling](https://jijmodeling-docs.jij-cloud.com/)】がある。

### Hamiltonian

- 定式化した目的関数、制約項を結合し、Hamiltonianと呼ばれる、本問題のエネルギーに対応する物理量を表現する。
- 各制約項には、重み係数としてλ(ラムダ)が掛け合わされる。制約項が0以外の値を返却(制約違反)した場合、係数が高い程、重いペナルティが課される。

$$
H = - \sum_{t \in_{T}, s \in_{S}} (C_{ts} * x_{ts})
+ \lambda_0 \sum_{t \in_{T}}(\sum_{s \in_{S}} x_{ts} - 1)^2
+ \lambda_1 \sum_{s \in_{S}}(\sum_{t \in_{T}} x_{ts} - Sn_{s})^2
+ \lambda_2 \sum_{t \in_{T}}(\sum_{s \in_{S}} NG_{ts} * x_{ts})
$$

In [13]:
# 講師t∈Tが講義s∈Sを担当するかどうかの目的変数
# 今回は38人9講義で38*9=342変数
x_ts = pq.Array.create(name='x', shape=(Tn, Sn), vartype='BINARY')

# Hamiltonianの構築
H = objective(x_ts)

H += pq.Placeholder(label='lambda_0') * pq.Constraint(
    hamiltonian=constraint_onlyone(x_ts), label='lambda_0')

H += pq.Placeholder(label='lambda_1') * pq.Constraint(
    hamiltonian=constraint_capacity(x_ts), label='lambda_1')

H += pq.Placeholder(label='lambda_2') * pq.Constraint(
    hamiltonian=constraint_unwanted(x_ts), label='lambda_2')

In [14]:
# 各制約項の重み係数を設定
feed_dict = {key: 100 for key in ['lambda_0', 'lambda_1', 'lambda_2']}
print(feed_dict)

{'lambda_0': 100, 'lambda_1': 100, 'lambda_2': 100}


### QUBO (Quadratic Unconstrained Binary Optimization)

- QUBOは、組合せ最適化問題の表現形式であり、(疑似)量子アニーリングで問題を解くことができるようになる。
- HamiltonianからQUBOへの変換は、PyQUBOのAPIを利用する。

In [15]:
# QUBOへの変換
model = H.compile()
qubo, _ = model.to_qubo(feed_dict=feed_dict)

## 5. (疑似)量子アニーリング

- ライブラリを利用して、(疑似)量子アニーリングを実行する。ここでは、【[OpenJij](https://github.com/OpenJij/OpenJij)】を利用する。
- 利用できるライブラリ・環境として、他に【[D-Wave](https://docs.ocean.dwavesys.com/en/stable/index.html)】がある。

In [16]:
# SQA(Simulated Quantum Annealing)実行モジュール
# num_reads: サンプリング回数, num_sweeps: サンプリング時間
sampler = oj.SQASampler(num_reads=100, num_sweeps=1000, trotter=20)

In [17]:
%%time
# (疑似)量子アニーリング実行
sampleset = sampler.sample_qubo(Q=qubo)

CPU times: user 1min 18s, sys: 321 ms, total: 1min 18s
Wall time: 41.1 s


In [18]:
# 実行結果をデコード
decoded = model.decode_sampleset(sampleset=sampleset, feed_dict=feed_dict)
# 最良解(最もエネルギーの小さい解)を抽出
best = min(decoded, key=lambda d: d.energy)
# 最良解が全制約を満たしているか確認
assert best.constraints(only_broken=True) == {}
# 最良解のエネルギー
print(best.energy)

-60.0


## 6. 可視化

- 最良解の結果をDataFrameに変換し、表示する。

In [19]:
df_x = pd.DataFrame(data=np.zeros(shape=(Tn, Sn), dtype=np.int),
                    index=T, columns=[chr(s+65) for s in S], dtype=int)
# 正規表現を用いて、最良解のサンプリング結果をDataFrameに格納
for key, value in best.sample.items():
  if value == 1:
    t_s = re.match(pattern='x\[(\d+)\]\[(\d+)\]', string=key)
    t, s = int(t_s.group(1)), int(t_s.group(2))
    df_x.iat[t, s] = value

In [20]:
# 講師 0～37 が講義 A～I の中から1つずつ受け持つ
display_html(df_x.style.applymap(lambda x: 'color: red' if x == 1 else 'color: gray')._repr_html_(), raw=True)

,A,B,C,D,E,F,G,H,I
0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,1
7,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0
9,0,0,0,0,1,0,0,0,0


## 7. 振り返り・展望

- 「スポット講義の講師の割り当て」というオリジナルな問題について、問題設定、定式化、構築、実行、評価という量子アニーリングの一連のタスクを実装することができた。
- 本問題の改良の余地として、講師の属性(ベテラン、若手など)を考慮した割り当ての仕組みを検討していきたい。